# Transaction Data Normalization & Migration Notebook

This notebook documents the process of cleaning, normalizing, and validating
personal financial transaction data collected from multiple monthly Excel files
throughout the year.

The original data sources contained inconsistent formats, including:
- mixed date representations (Excel serial dates, EU/US date strings),
- missing or extra columns across months,
- partially empty rows,
- inconsistent numeric formats for transaction amounts.

The goal of this notebook was to:
- explore and understand the structure of the raw Excel data,
- iteratively normalize dates, headers, and numeric values,
- validate schema consistency across all months,
- ensure that no transactions were lost during transformation,
- prepare the data for safe insertion into a SQLite database.

This notebook served as an experimental and diagnostic environment:
all outputs have been cleared to keep it readable and suitable for version control.
The final, production-ready logic derived from this notebook is implemented
in a standalone Python migration script.

In [ ]:
import pandas as pd

In [ ]:
def normalize_dates_mmddyyyy(
    series: pd.Series,
    expected_month: int,
    expected_year: int | None = None,
) -> pd.Series:
    def parse_one(v):
        if pd.isna(v):
            return pd.NaT

        # Already a datetime-like?
        if isinstance(v, (pd.Timestamp, )):
            ts_candidates = [v]
        else:
            s = str(v).strip()
            # Try both interpretations
            ts_dayfirst  = pd.to_datetime(s, dayfirst=True,  errors="coerce")
            ts_monthfirst = pd.to_datetime(s, dayfirst=False, errors="coerce")
            ts_candidates = [ts_dayfirst, ts_monthfirst]

        ts_candidates = [t for t in ts_candidates if not pd.isna(t)]
        if not ts_candidates:
            return pd.NaT

        # Optional year filter (helps if your data is only 2025)
        if expected_year is not None:
            year_ok = [t for t in ts_candidates if t.year == expected_year]
            if year_ok:
                ts_candidates = year_ok

        # Prefer the one that lands in the expected month (January = 1)
        for t in ts_candidates:
            if t.month == expected_month:
                return t

        # Fallback: return the first valid parse
        return ts_candidates[0]

    parsed = series.apply(parse_one)
    return parsed.dt.strftime("%d-%m-%Y")

JANUARY

In [ ]:
from pathlib import Path
import pandas as pd

# Folder with monthly Excel files
DATA_DIR = Path("processed_data")

def load_excel_files(folder: Path) -> dict[str, pd.DataFrame]:
    dfs = {}

    for file in sorted(folder.glob("*.xlsx")):
        key = file.stem.lower()  
        df = pd.read_excel(file)

        dfs[key] = df

        print(f"Loaded {file.name}: {df.shape[0]} rows, {df.shape[1]} columns")

    return dfs

In [ ]:
from datetime import datetime, date
import pandas as pd

def normalize_date(value) -> str | None:
    """
    Normalize date into DD-MM-YYYY.

    Accepts:
    - string dates (DD/MM/YYYY, D/M/YY, etc.)
    - datetime / date objects
    - NaN / NaT

    Returns:
    - formatted string or None
    """

    if pd.isna(value):
        return None

    # Already a datetime/date object
    if isinstance(value, (datetime, date)):
        return value.strftime("%d-%m-%Y")

    # String input
    if isinstance(value, str):
        value = value.strip()

        formats = ("%d/%m/%Y", "%d/%m/%y")

        for fmt in formats:
            try:
                return datetime.strptime(value, fmt).strftime("%d-%m-%Y")
            except ValueError:
                continue

    raise ValueError(f"Unsupported date value: {value} ({type(value)})")

In [ ]:
dataframes = load_excel_files(Path("/Users/test/Documents/Documents/Personal/projects/finance-tracker-webapp/data-migration/processed_data"))

In [ ]:
january_df = dataframes["january"]

In [ ]:
january_df["DATE"] = january_df["DATE"].apply(normalize_date)

In [ ]:
january_df = january_df.rename(columns={
    "DATE": "date",
    "PAYMENT METHOD": "account",
    "DESCRIPTION": "description",
    "SUM": "sum",
    "CATEGORY": "category",
    "NOTES": "notes",
})

In [ ]:
january_df["date"] = normalize_dates_mmddyyyy(
    january_df["date"],
    expected_month=1,
    expected_year=2025
)

In [ ]:
january_df

In [ ]:
january_df.to_csv("./normalized/january.csv", index=False, encoding='utf-8')

February

In [ ]:
february_df = dataframes["february"]

In [ ]:
february_df["DATE"] = (
    pd.to_datetime(february_df["DATE"], format="%d/%m/%y")
    .dt.strftime("%d-%m-%Y")
)

In [ ]:
february_df = february_df.rename(columns={
    "DATE": "date",
    "PAYMENT METHOD": "account",
    "DESCRIPTION": "description",
    "SUM": "sum",
    "CATEGORY": "category",
    "NOTES": "notes",
})

In [ ]:
february_df

In [ ]:
february_df['date'] = normalize_dates_mmddyyyy(
    february_df["date"],
    expected_month=2,
    expected_year=2025
)

In [ ]:
february_df.to_csv("./normalized/february.csv", index=False, encoding='utf-8')

MARCH

In [ ]:
march_df = dataframes["march"]

In [ ]:
march_df

In [ ]:
march_df["DATE"] = pd.to_datetime(march_df["DATE"], format="%d/%m/%y").dt.strftime("%d-%m-%Y")

In [ ]:
march_df = march_df.rename(columns={
    "DATE": "date",
    "PAYMENT METHOD": "account",
    "DESCRIPTION": "description",
    "SUM": "sum",
    "CATEGORY": "category",
    "NOTES": "notes",
})

In [ ]:
march_df.head(30)

In [ ]:
march_df.to_csv("./normalized/march.csv", index=False, encoding='utf-8')

April

In [ ]:
april_df = dataframes["april"]

In [ ]:
april_df["DATE"] = (
    pd.to_datetime(april_df["DATE"], format="%d/%m/%y")
    .dt.strftime("%d-%m-%Y")
)

In [ ]:
april_df

In [ ]:
def normalize_mixed_date(value):
    # Case 1: Excel serial date (int / float)
    if isinstance(value, (int, float)):
        return pd.to_datetime(value, unit="D", origin="1899-12-30").strftime("%d-%m-%Y")

    # Case 2: EU string date (DD/MM/YYYY)
    if isinstance(value, str):
        return pd.to_datetime(value, format="%d/%m/%Y").strftime("%d-%m-%Y")

    return None


In [ ]:
april_df = april_df.rename(columns={
    "Date": "date",
    "Description": "description",
    "Sum": "sum",
    "Category": "category",
    "Notes": "notes",
})

In [ ]:
import numpy as np
april_df["account"] = np.nan

In [ ]:
april_df

In [ ]:
april_df.to_csv("./normalized/april.csv", index=False, encoding='utf-8')

May

In [ ]:
may_df = dataframes['may']

In [ ]:
may_df['txn_date'] = may_df['txn_date'].apply(normalize_mixed_date)

In [ ]:
may_df["account"] = np.nan

In [ ]:
may_df = may_df.rename(columns={
    "txn_date": "date",
    "amount": "sum",
    "note": "notes"
})

In [ ]:
may_df['date'] = normalize_dates_mmddyyyy(
    may_df["date"],
    expected_month=5,
    expected_year=2025
)

In [ ]:
may_df['date'] = normalize_dates_mmddyyyy(
    may_df["date"],
    expected_month=5,
    expected_year=2025
)

In [ ]:
may_df

In [ ]:
may_df.to_csv("./normalized/may.csv", index=False, encoding='utf-8')

June

In [ ]:
june_df = dataframes["june"]

In [ ]:
june_df = june_df.dropna(how="all")

In [ ]:
june_df['txn_date'] = pd.to_datetime(june_df["txn_date"]).dt.strftime("%d-%m-%Y")

In [ ]:
june_df = june_df.drop(columns=['type'])

In [ ]:
june_df['date'] = normalize_dates_mmddyyyy(
    june_df["date"],
    expected_month=6,
    expected_year=2025
)

In [ ]:
june_df

In [ ]:
june_df = june_df.rename(columns={
    "txn_date": "date",
    "amount": "sum",
    "note":"notes"
})

In [ ]:
june_df.to_csv("./normalized/june.csv", index=False, encoding='utf-8')

July

In [ ]:
july_df = dataframes['july']

In [ ]:
from datetime import datetime, date

def normalize_any_date(v):
    if pd.isna(v):
        return None

    # already parsed datetime/date
    if isinstance(v, (datetime, date)):
        return v.strftime("%d-%m-%Y")

    # excel serial (int/float)
    if isinstance(v, (int, float)):
        return pd.to_datetime(v, unit="D", origin="1899-12-30").strftime("%d-%m-%Y")

    # strings (like "13/07/2025" or even "45664" as string)
    if isinstance(v, str):
        s = v.strip()

        if s.isdigit():  # handles "45664" stored as text
            n = int(s)
            return pd.to_datetime(n, unit="D", origin="1899-12-30").strftime("%d-%m-%Y")

        return datetime.strptime(s, "%d/%m/%Y").strftime("%d-%m-%Y")

    raise ValueError(f"Unsupported date value: {v} ({type(v)})")

july_df["date"] = july_df["date"].apply(normalize_any_date)

In [ ]:
july_df

In [ ]:
july_df['date'] = normalize_dates_mmddyyyy(
    july_df["date"],
    expected_month=7,
    expected_year=2025
)

In [ ]:
july_df.to_csv("./normalized/july.csv", index=False, encoding='utf-8')

August

In [ ]:
august_df = dataframes['august']

In [ ]:
august_df['date'] = august_df['date'].apply(normalize_any_date)

In [ ]:
august_df

In [ ]:
august_df['date'] = normalize_dates_mmddyyyy(
    august_df["date"],
    expected_month=8,
    expected_year=2025
)

In [ ]:
august_df.to_csv("./normalized/august.csv", index=False, encoding='utf-8')

September

In [ ]:
september_df = dataframes["september"]

In [ ]:
september_df['txn_date'] = september_df['txn_date'].apply(normalize_any_date)

In [ ]:
september_df = september_df.rename(columns={
    "txn_date": "date",
    "amount": "sum",
})

In [ ]:
september_df['date'] = normalize_dates_mmddyyyy(
    september_df["date"],
    expected_month=9,
    expected_year=2025
)

In [ ]:
september_df.to_csv("./normalized/september.csv", index=False, encoding='utf-8')

October

In [ ]:
october_df = dataframes['october']

In [ ]:
october_df['date'] = october_df['date'].apply(normalize_any_date)

In [ ]:
october_df['date'] = normalize_dates_mmddyyyy(
    october_df["date"],
    expected_month=10,
    expected_year=2025
)

In [ ]:
october_df.to_csv("./normalized/october.csv", index=False, encoding='utf-8')

November

In [ ]:
november_df = dataframes['november']

In [ ]:
november_df['txn_date'] = november_df['txn_date'].apply(normalize_any_date)

In [ ]:
november_df = november_df.rename(columns={
    "txn_date": "date",
    "amount": "sum",
})

In [ ]:
november_df['date'] = normalize_dates_mmddyyyy(
    november_df["date"],
    expected_month=11,
    expected_year=2025
)

In [ ]:
november_df.to_csv("./normalized/november.csv", index=False, encoding='utf-8')

------------------

In [ ]:
from pathlib import Path
import pandas as pd
import re

REQUIRED_COLS = {"date", "description", "sum", "category", "notes", "account"}
DATE_RE = re.compile(r"^\d{2}-\d{2}-\d{4}$")

def validate_normalized_csvs(folder: str | Path) -> dict:
    folder = Path(folder)
    csv_files = sorted(folder.glob("*.csv"))

    if not csv_files:
        raise FileNotFoundError(f"No .csv files found in: {folder}")

    report = {
        "files_checked": 0,
        "header_ok": True,
        "date_ok": True,
        "sum_ok": True,
        "issues": [],   # list of dicts
    }

    baseline_cols = None

    for f in csv_files:
        df = pd.read_csv(f)
        report["files_checked"] += 1

        cols = set(df.columns.str.strip().str.lower())
        missing = REQUIRED_COLS - cols
        extra = cols - REQUIRED_COLS

        if missing:
            report["header_ok"] = False
            report["issues"].append({
                "file": f.name,
                "type": "missing_columns",
                "details": sorted(missing),
            })

        # optional: flag extra columns (not fatal)
        if extra:
            report["issues"].append({
                "file": f.name,
                "type": "extra_columns",
                "details": sorted(extra),
            })

        # check all files have same headers (ignoring order)
        if baseline_cols is None:
            baseline_cols = cols
        elif cols != baseline_cols:
            report["header_ok"] = False
            report["issues"].append({
                "file": f.name,
                "type": "header_mismatch_vs_first_file",
                "details": {
                    "missing_vs_first": sorted(baseline_cols - cols),
                    "extra_vs_first": sorted(cols - baseline_cols),
                },
            })

        # normalize column names access
        df.columns = df.columns.str.strip().str.lower()

        # --- DATE checks ---
        if "date" in df.columns:
            # ensure string and basic format DD-MM-YYYY
            date_str = df["date"].astype(str).str.strip()
            bad_format = df[~date_str.fillna("").str.match(DATE_RE)]
            # allow empty date only if whole row is empty-ish; but for DB it's usually not ok:
            bad_format = bad_format[date_str.notna() & (date_str != "nan")]

            if len(bad_format) > 0:
                report["date_ok"] = False
                report["issues"].append({
                    "file": f.name,
                    "type": "bad_date_format",
                    "details": f"{len(bad_format)} rows not matching DD-MM-YYYY (example: {bad_format['date'].iloc[0]!r})",
                })

            # parseability check (strict)
            parsed = pd.to_datetime(date_str, format="%d-%m-%Y", errors="coerce")
            bad_parse = df[parsed.isna() & ~(date_str.isna() | (date_str == "nan"))]
            if len(bad_parse) > 0:
                report["date_ok"] = False
                report["issues"].append({
                    "file": f.name,
                    "type": "unparseable_dates",
                    "details": f"{len(bad_parse)} rows unparseable (example: {bad_parse['date'].iloc[0]!r})",
                })

        # --- SUM checks ---
        if "sum" in df.columns:
            # handle "1,23" -> "1.23" and strip spaces
            sum_clean = (
                df["sum"]
                .astype(str)
                .str.replace(" ", "", regex=False)
                .str.replace(",", ".", regex=False)
                .str.strip()
            )
            sum_num = pd.to_numeric(sum_clean, errors="coerce")
            bad_sum = df[sum_num.isna() & ~(sum_clean.isna() | (sum_clean == "nan") | (sum_clean == ""))]

            if len(bad_sum) > 0:
                report["sum_ok"] = False
                report["issues"].append({
                    "file": f.name,
                    "type": "non_numeric_sum",
                    "details": f"{len(bad_sum)} rows have non-numeric sum (example: {bad_sum['sum'].iloc[0]!r})",
                })

        # --- critical empties (warnings, not fatal) ---
        critical = []
        for c in ["date", "description", "sum"]:
            if c in df.columns:
                n_missing = df[c].isna().sum()
                if n_missing:
                    critical.append(f"{c}: {n_missing}")
        if critical:
            report["issues"].append({
                "file": f.name,
                "type": "missing_critical_fields_warning",
                "details": ", ".join(critical),
            })

    return report

In [ ]:
report = validate_normalized_csvs("./normalized")
report

In [ ]:
from pathlib import Path
import pandas as pd

normalized_dir = Path("./normalized")

dfs = []
for csv_file in sorted(normalized_dir.glob("*.csv")):
    df = pd.read_csv(csv_file)
    df["source_file"] = csv_file.name  # optional but very useful
    dfs.append(df)

all_df = pd.concat(dfs, ignore_index=True)

In [ ]:
all_df

In [ ]:
all_df[(df["date"] >= "2025-03-01") & (df["date"] < "2025-04-01"), "amount_eur"].sum()

In [ ]:
df.loc[(df["date"] >= "2025-03-01") & (df["date"] < "2025-04-01")]